---
## Simple generator example reading midi files
### Monday: 26-02-2024

* Read the MIDI file normally
    * We are given a list with all midi elements

* Read the MIDI file using the generator 
    * We are "`yield`ed" each element individually
    * We can get elements with `next()` or with a `loop`

---

In [3]:
#To help us import from outside folder
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))
from midiHandler import MidiHandler
from Utils import Utils as utils

midi_file_path = "../data/midi/test.mid"

midi = MidiHandler()

# Read the MIDI file normally
data = midi.readMidi(midi_file_path)
#print(data) # Lets not print to save notebook space

# Read the MIDI file using the generator 
data = midi.readMidiGenerator(midi_file_path)
for d in data:
    #print(d, end=' ') # Lets not print to save notebook space
    pass


---
## Lets get some random Midi sections
### Friday: 01-03-2024

* Read the MIDI file and retrieve the notes we want
   * Handle 1 file with and without generators

* Read the MIDI file using generators and threads
    * Use all cpu cores with yield generators
    
* Compare the differences between using generators and not
---

In [2]:
from concurrent.futures import ThreadPoolExecutor

directory = "../data/midi/pop/"
file_paths = [os.path.join(directory, file) for file in os.listdir(directory)]

def getNotes(midi_file_path):
    midi = MidiHandler()
    notes = midi.getNotes(midi_file_path)
    #print(notes)
    print("Total notes retrieved: ", len(notes))

def getNotesGenerator(midi_file_path):
    midi = MidiHandler()
    notes = midi.getNotesGenerator(midi_file_path)
    count = 0
    for n in notes:
        #print(n)
        count += 1
    print("Total notes retrieved using generator: ", count)

def getNotesThreaded(directory):
    file_paths = [os.path.join(directory, file) for file in os.listdir(directory)]
    num_workers = min(os.cpu_count(), len(file_paths))
    print("Number of workers: ", num_workers)

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(midi.getNotes, file_path) for file_path in file_paths]
        count = 0
        for future in futures:
            if future.result() is not None:
                notes = future.result()
                #print(notes)
                count += len(notes)
    print("Total notes retrieved using threaded: ", count)

def getNotesThreadedGenerator(directory):
    file_paths = [os.path.join(directory, file) for file in os.listdir(directory)]
    num_workers = min(os.cpu_count(), len(file_paths)) 
    print("Number of workers: ", num_workers)

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(midi.getNotesGenerator, file_path) for file_path in file_paths]
        count = 0
        for future in futures:
            notes = future.result()
            if notes is not None:
                for n in notes:
                    #print(n)
                    count += 1
    print("Total notes retrieved using threaded generator: ", count)

print("\n"+"--Using normal function")
utils.measure_function(getNotes, midi_file_path)

print("\n"+"--Using generator function")
utils.measure_function(getNotesGenerator, midi_file_path)

print("Total MIDI files: ", len(file_paths))

print("\n"+"--Using threaded function")
utils.measure_function(getNotesThreaded, directory)

print("\n"+"--Using threaded generator function")
utils.measure_function(getNotesThreadedGenerator, directory)


Total MIDI files:  2622

--Using normal function
Total notes retrieved:  5
Memory used: 0.0078125 MB
Time taken: 0.00099945068359375 seconds


--Using generator function
Total notes retrieved using generator:  5
Memory used: 0.0 MB
Time taken: 0.0009839534759521484 seconds


--Using threaded function
Total notes retrieved using threaded:  1510
Memory used: 60.66015625 MB
Time taken: 290.51070737838745 seconds


--Using threaded generator function
Total notes retrieved using threaded generator:  1510
Memory used: -11.16796875 MB
Time taken: 181.09490942955017 seconds

